# AIPI 590 - XAI | Interpretable AI
### Understand a dataset and develop interpretable models to make predictions
### Mariam Khan

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mkhan13/XAI/blob/interpretable_ml_assignment/khan_interpretable_ml.ipynb)

In [ ]:
#installs
!pip install kaggle

In [ ]:
#imports
from google.colab import userdata
import os

In [ ]:
# Set Kaggle credentials
os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')

# Download dataset
!kaggle datasets download -d blastchar/telco-customer-churn
!unzip telco-customer-churn.zip -d telco_data

In [ ]:
import pandas as pd
df = pd.read_csv('telco_data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

### **1. Exploratory Data Analysis to check Assumptions**
Perform an exploratory analysis of the dataset to understand the relationships between different features and the target variable (churn). Use appropriate visualizations and statistical methods to determine whether assumptions about linear, logistic, and GAM models are met.

###**2. Linear Regression**
Treat the churn variable as a continuous variable (e.g., O for staying, 1 for churning) and build a linear regression model to predict churn.
Interpret the coefficients and assess the model's performance.


###**3. Logistic Regression**
Treat churn as a binary variable and build a logistic regression model to predict the probability of churn. Interpret the coefficients.


###**4. Generalized Additive Model (GAM)**
Build a GAM to model the non-linear relationships between customer features and churn. Interpret the GAM model.

### **5. Model Comparison**
Compare the performance and interpretability of the different models you built. Discuss the strengths and weaknesses of each approach and provide recommendations for which model(s) the telecommunications company should use to address their customer churn problem.